In [3]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [4]:
conf = Config()

conf.dataset = 'nyc'
conf.post_value_updates()

In [5]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
model

TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBTimeWeighted(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
            )
            (linear1): Linear(in_features=512, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=512, bias=True)
            (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): 

In [7]:
# from ah_databricks_data_loader import DatabricksDataLoader

# # Instantiate the DatabricksDataLoader.
# ddl = DatabricksDataLoader()

# # Load data.
# test_df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_traj_data_v18")

In [8]:
# test_df.count()

In [9]:
# test_df.show(5)

In [10]:
# test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18")

In [11]:
# test_df.count()

In [12]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

Total records in repartitioned files: 1445257


In [13]:
df_list[0].head()

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
0,5240,2025-05-12,"[2025-05-12T11:23:06.000000000, 2025-05-12T11:...","[[-73.8074343, 40.909735], [-73.9277252, 40.81...",Trader Joe's,1,1478.72,train,30,"[[-8216206.003033994, 4999036.699901179], [-82...",0,"[68, 69, 71, 75, 76, 78, 79, 83, 89, 91, 98, 9..."
1,34299,2024-12-06,"[2024-12-06T00:01:53.000000000, 2024-12-06T00:...","[[-74.0592517, 40.7562114], [-74.0598949, 40.7...",u s postal service,0,3026.97,train,64,"[[-8244238.187774881, 4976449.130531165], [-82...",4,"[0, 1, 2, 4, 5, 7, 8, 9, 11, 12, 14, 15, 18, 2..."
2,34299,2025-02-15,"[2025-02-15T00:00:06.000000000, 2025-02-15T00:...","[[-74.0596466, 40.7565547], [-74.0592928, 40.7...",u s postal service,0,4129.42,train,70,"[[-8244282.147841794, 4976499.581186923], [-82...",5,"[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 1..."
3,35404,2024-11-08,"[2024-11-08T00:34:19.000000000, 2024-11-08T06:...","[[-73.9452781, 40.7803969], [-73.9452432, 40.7...",Con edison,0,0.00,train,29,"[[-8231550.704659003, 4980004.019033388], [-82...",4,"[3, 36, 38, 39, 40, 41, 42, 44, 45, 46, 48, 51..."
4,54320,2025-05-06,"[2025-05-06T02:00:27.000000000, 2025-05-06T07:...","[[-73.9059497, 40.8341471], [-73.9078743, 40.8...",Sterling Nat Bnk,0,642.83,train,28,"[[-8227172.6871972885, 4987909.091900762], [-8...",1,"[12, 47, 48, 50, 52, 54, 55, 56, 57, 59, 60, 6..."


In [14]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [15]:
# userids = test_df['userid'].unique()

In [16]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/nyc_time_weighted_new_aug/nyc_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_12407/2123240561.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBTimeWeighted(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
            )
            (linear1): Linear(in_features=512, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=512, bias=True)
            (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): 

In [17]:
from utils.traj import *
import pickle

from torch.nn.utils.rnn import pad_sequence
embs = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_embdim512_embs.pkl", 'rb')).to('cpu').detach() # tensor
cellspace = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_cellspace.pkl", 'rb'))

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell, traj_p, traj_timedelta = zip(*[merc2cell2(l,t, cellspace) for l,t in zip( traj, time_indices)])
    traj_emb_p = [torch.tensor(generate_spatial_features(t, cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell = [embs[list(t)] for t in traj_cell]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_cell)), dtype = torch.long, device = device)
    time_deltas = pad_sequence([torch.log(torch.tensor(t)) for t in traj_timedelta], batch_first=False, padding_value=0).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, time_deltas)
    return traj_embs, traj_cell, time_deltas

def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size]
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [18]:
from sklearn.metrics.pairwise import cosine_similarity


In [19]:
df_list[0]['time_index_list'].values[0]

array([ 68,  69,  71,  75,  76,  78,  79,  83,  89,  91,  98,  99, 103,
       105, 109, 112, 113, 115, 117, 118, 120, 122, 123, 125, 126, 128,
       129, 130, 131, 133], dtype=int32)

In [20]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

Total unique userids: 25831
Total unique employernames: 13911


In [21]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['timestamps'].values
    return traj, time_indices

In [22]:
def get_min_max_xy(traj):
    min_x = min([p[0] for p in traj])
    max_x = max([p[0] for p in traj])
    min_y = min([p[1] for p in traj])
    max_y = max([p[1] for p in traj])
    return min_x, max_x, min_y, max_y

def calculate_dx_dy(k, target_min_x, target_min_y, min_x, max_x, min_y, max_y):
    dx = target_min_x+ (k - max_x - min_x) / 2
    dy = target_min_y+ (k - max_y - min_y) / 2
    return dx, dy

def transform_traj(traj, dx, dy):
    new_traj = [[p[0] + dx, p[1] + dy] for p in traj]
    return new_traj

def normalize(trajs, cellspace):
    # trajs: list of [[lon, lat], [,], ...]

    # 1. augment the input traj in order to form 2 augmented traj views
    # 2. convert augmented trajs to the trajs based on mercator space by cells
    # 3. read cell embeddings and form batch tensors (sort, pad)

    trajs1, trajs2 = [], []
    time_indices1, time_indices2 = [], []
    final_min_x = 1000000000
    final_min_y = 1000000000
    final_max_x = -1000000000
    final_max_y = -1000000000
    max_len_meters = min(cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min)
    indices_to_omit = []
    for i, l in enumerate(trajs):
        min_x, max_x, min_y, max_y = get_min_max_xy(l)
        # if final_max_x - min_x> max_len_meters or final_max_y-min_y> max_len_meters or max_x - final_min_x> max_len_meters or max_y - final_min_y> max_len_meters or max_x-min_x> max_len_meters or max_y-min_y> max_len_meters:
        #     indices_to_omit.append(i)
        #     continue
        final_min_x = min(final_min_x, min_x)
        final_min_y = min(final_min_y, min_y)
        final_max_x = max(final_max_x, max_x)
        final_max_y = max(final_max_y, max_y)
    
    if max(final_max_x-final_min_x, final_max_y-final_min_y)>max_len_meters:
        # print(max(final_max_x-final_min_x, final_max_y-final_min_y), max_len_meters)
        return None
    dx, dy = calculate_dx_dy(max_len_meters, cellspace.x_min, cellspace.y_min, final_min_x, final_max_x, final_min_y, final_max_y)
    # print(final_min_x, final_max_x, final_min_y, final_max_y)
    # print(dx, dy)
    for i,l in enumerate(trajs):
        if i in indices_to_omit:
            continue
        new_l = transform_traj(l, dx, dy)
        trajs1.append(new_l)
    return trajs1

In [23]:
def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [25]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                output_dict[(userid, employer, partition)] = 2
            else:
                traj, time_indices = get_traj_and_time_data(part_data)
                embs, _, _ = infer(traj, time_indices)
                db_scan = apply_dbscan(embs.detach().cpu().numpy(), target_min_similarity=0.85)
                labels = db_scan.labels_                      # shape: (n_samples,)
                cluster_ids = [c for c in np.unique(labels) if c != -1]
                if len(cluster_ids)>0:
                    pred_label = 1
                else:
                    pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label

for userid in tqdm(unique_userids):
    get_gt_and_pred_label(userid)


        


100%|██████████| 25831/25831 [18:05<00:00, 23.80it/s] 


In [26]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/nyc_time_weighted_new_aug/nyc_test_hyp1_output_dict.pkl", 'wb'))

In [27]:
output_dict

{(5240, "Trader Joe's", 1): 1,
 (5240, "Trader Joe's", 0): 1,
 (34299, 'u s postal service', 0): 1,
 (34299, 'u s postal service', 1): 1,
 (35404, 'Con edison', 0): 1,
 (54320, 'Sterling Nat Bnk', 0): 0,
 (54320, 'Medix Staffing', 0): 2,
 (87926, 'Bloomberg lp', 0): 1,
 (87926, '474-M.S. SERVICE', 0): 1,
 (104834, 'NYC Department of Finance', 0): 1,
 (104834, 'NYC Department of Finance', 1): 1,
 (104834, 'city of new york', 0): 1,
 (105722, 'Mount sinai hospital', 0): 2,
 (105722, 'Mount sinai hospital', 1): 1,
 (105722, 'mt. sinai', 0): 2,
 (119316, 'MVP Hospitality', 0): 1,
 (123076, 'The Capital Markets Company', 0): 2,
 (123076, 'The Capital Markets Company', 1): 2,
 (126387, 'United Airlines', 0): 1,
 (126387, 'United Airlines', 1): 1,
 (131307, 'American international security llc', 0): 2,
 (131458, 'Downtown Bronx ASC', 0): 1,
 (131458, 'Downtown Bronx ASC', 1): 1,
 (134952, 'AT&T', 0): 1,
 (134952, 'AT&T', 1): 1,
 (147693, 'Women In Need, Inc. ', 0): 1,
 (150194, 'Nyc health an

In [28]:
count_dict = {}
for k,v in output_dict.items():
    if v in count_dict:
        count_dict[v]+=1
    else:
        count_dict[v]=1
count_dict

{1: 15398, 0: 5745, 2: 14285}

In [47]:
output_dict[(19543365, 'AMAZON.COM SVCS DIRECT DEP', 0)]

1

In [30]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part_fil = df_part_fil[df_part_fil['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part_fil

In [48]:
df_part = get_data_userid_employer_partition(19543365, 'AMAZON.COM SVCS DIRECT DEP', 0)
df_part.sort_values('traj_date')

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
2,19543365,2024-07-01,"[2024-07-01T00:06:00.000000000, 2024-07-01T00:...","[[-73.96185179340952, 40.657196044921875], [-7...",AMAZON.COM SVCS DIRECT DEP,0,1005.80,train,27,"[[-8233395.6797699155, 4961908.89507006], [-82...",0,"[0, 4, 5, 7, 8, 44, 52, 73, 74, 75, 76, 77, 79..."
18,19543365,2024-07-02,"[2024-07-02T00:02:10.000000000, 2024-07-02T00:...","[[-73.80742325042677, 40.70025634765625], [-73...",AMAZON.COM SVCS DIRECT DEP,0,1005.80,train,17,"[[-8216204.773001128, 4968229.570674998], [-82...",1,"[0, 1, 2, 12, 55, 75, 76, 77, 78, 79, 82, 120,..."
27,19543365,2024-07-03,"[2024-07-03T00:31:09.000000000, 2024-07-03T00:...","[[-73.95875564983938, 40.65576171875], [-73.92...",AMAZON.COM SVCS DIRECT DEP,0,1005.80,train,20,"[[-8233051.018644264, 4961698.425438787], [-82...",2,"[3, 4, 5, 6, 78, 80, 81, 83, 85, 86, 89, 90, 9..."
48,19543365,2024-07-04,"[2024-07-04T00:01:04.000000000, 2024-07-04T00:...","[[-73.9721575479286, 40.65118408203125], [-73....",AMAZON.COM SVCS DIRECT DEP,0,1005.80,train,16,"[[-8234542.91111522, 4961026.744112313], [-823...",3,"[0, 1, 2, 5, 6, 43, 55, 56, 58, 59, 62, 94, 96..."
1,19543365,2024-07-08,"[2024-07-08T00:11:42.000000000, 2024-07-08T00:...","[[-73.96151477099583, 40.6807861328125], [-73....",AMAZON.COM SVCS DIRECT DEP,0,789.41,train,19,"[[-8233358.162606438, 4965371.09826278], [-822...",0,"[1, 2, 3, 5, 74, 75, 76, 78, 80, 92, 95, 96, 1..."
51,19543365,2024-07-09,"[2024-07-09T00:00:55.000000000, 2024-07-09T00:...","[[-73.80804227522638, 40.699676513671875], [-7...",AMAZON.COM SVCS DIRECT DEP,0,789.41,train,24,"[[-8216273.68252661, 4968144.431685407], [-821...",1,"[0, 1, 2, 60, 73, 74, 76, 77, 78, 107, 115, 11..."
50,19543365,2024-07-11,"[2024-07-11T00:17:00.000000000, 2024-07-11T00:...","[[-73.96240784043053, 40.657958984375], [-73.9...",AMAZON.COM SVCS DIRECT DEP,0,789.41,train,23,"[[-8233457.578641151, 4962020.848845211], [-82...",3,"[1, 2, 3, 4, 44, 45, 73, 74, 75, 76, 77, 78, 8..."
46,19543365,2024-07-12,"[2024-07-12T00:03:13.000000000, 2024-07-12T00:...","[[-73.94094956152418, 40.662811279296875], [-7...",AMAZON.COM SVCS DIRECT DEP,0,779.91,train,25,"[[-8231068.853959995, 4962732.904823712], [-82...",4,"[0, 1, 2, 5, 6, 46, 49, 51, 52, 54, 55, 56, 57..."
9,19543365,2024-07-15,"[2024-07-15T00:03:51.000000000, 2024-07-15T00:...","[[-73.9970618939997, 40.72509765625], [-73.962...",AMAZON.COM SVCS DIRECT DEP,0,779.91,train,19,"[[-8237315.250238393, 4971877.800520812], [-82...",0,"[0, 2, 73, 76, 86, 88, 89, 91, 92, 111, 112, 1..."
16,19543365,2024-07-16,"[2024-07-16T00:00:52.000000000, 2024-07-16T00:...","[[-73.80765027215246, 40.70025634765625], [-73...",AMAZON.COM SVCS DIRECT DEP,0,779.91,train,22,"[[-8216230.04494403, 4968229.570674998], [-821...",1,"[0, 1, 11, 73, 75, 76, 77, 78, 80, 81, 120, 12..."


In [49]:
import numpy as np

def cosine_similarity_matrix(X, eps=1e-8, dtype=np.float32):
    X = np.asarray(X, dtype=dtype)
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    Xn = X / np.maximum(norms, eps)
    return Xn @ Xn.T

In [50]:
def get_cluster(part_df):
    traj , time_indices= get_traj_and_time_data(part_df)
    embs, cell, time_delta = infer(traj,time_indices)
    db_scan = apply_dbscan(embs.detach().cpu().numpy(), target_min_similarity=0.85)
    labels = db_scan.labels_                      # shape: (n_samples,)
    cluster_ids = [c for c in np.unique(labels) if c != -1]
    for i in part_df.index:
        print(labels[i], part_df.iloc[i]['traj_date'])

In [51]:
def get_similarity(part_df):
    traj_dates = list(part_df['traj_date'])
    traj , time_indices= get_traj_and_time_data(part_df)
    # traj = normalize(traj, cellspace)
    # print(traj)
    embs, cell, time_delta = infer(traj,time_indices)
    similarity = cosine_similarity_matrix(embs.detach().cpu().numpy())


    return similarity, cell, time_delta


In [52]:
import datetime

In [53]:
get_cluster(df_part)

-1 2024-11-26
0 2024-07-08
-1 2024-07-01
-1 2024-08-23
0 2024-09-11
0 2024-07-30
-1 2024-11-18
0 2024-08-27
0 2024-11-27
0 2024-07-15
0 2024-12-09
-1 2024-11-19
0 2024-09-17
0 2024-10-23
0 2024-10-30
0 2024-10-25
-1 2024-07-16
0 2024-10-29
0 2024-07-02
0 2024-07-22
0 2024-11-08
0 2024-10-31
-1 2024-11-13
0 2024-08-02
0 2024-12-18
0 2024-09-27
0 2024-10-24
0 2024-07-03
0 2024-12-02
0 2024-11-29
-1 2024-07-24
0 2024-07-17
-1 2024-11-14
0 2024-10-18
0 2024-09-06
0 2024-09-13
0 2024-11-01
0 2024-10-10
0 2024-12-05
0 2024-09-02
0 2024-07-25
0 2024-08-16
0 2024-07-31
0 2024-09-16
-1 2024-11-15
0 2024-08-15
-1 2024-07-12
0 2024-09-09
0 2024-07-04
-1 2024-08-08
0 2024-07-11
0 2024-07-09
0 2024-10-03
0 2024-11-20
-1 2024-10-28


In [81]:
df_fil= df_part[df_part['traj_date'].isin([datetime.date(2024,7,11), datetime.date(2024,7,4)])]
sim, cell, timedelta = get_similarity(df_fil)

In [82]:
sim

array([[1.0000002 , 0.87352514],
       [0.87352514, 0.99999964]], dtype=float32)

In [83]:
cell

([34030,
  36434,
  54069,
  63548,
  60596,
  61112,
  58229,
  45088,
  40877,
  35348,
  35611,
  35346,
  36408,
  33268,
  30617,
  11919],
 [35349,
  38772,
  44564,
  54814,
  60596,
  60592,
  59012,
  50083,
  44562,
  41141,
  37983,
  35349,
  35084,
  35347,
  35607,
  35346,
  35347,
  35083,
  34821,
  35084,
  34820,
  35348,
  35349])

In [84]:
timedelta

tensor([[2.4539, 2.4610],
        [2.4380, 2.4394],
        [3.5086, 2.4582],
        [2.5001, 5.9744],
        [5.9169, 2.8134],
        [4.8088, 5.6413],
        [2.5110, 2.4277],
        [2.5271, 2.4423],
        [2.7736, 2.4409],
        [3.2878, 2.4054],
        [5.7858, 2.4365],
        [3.0501, 2.5164],
        [2.7224, 2.7014],
        [2.6271, 2.6810],
        [2.5310, 2.4009],
        [0.0000, 5.7885],
        [0.0000, 2.6810],
        [0.0000, 4.0067],
        [0.0000, 2.8439],
        [0.0000, 2.7547],
        [0.0000, 2.5777],
        [0.0000, 5.0971],
        [0.0000, 0.0000]], device='cuda:0')

In [85]:
timedelta = timedelta.cpu().numpy()

In [86]:

sorted_indices_0 = list(np.argsort(timedelta[:,0]))[::-1]

cell_fil_0 = np.array(cell[0])[sorted_indices_0[:10]]
timestamps_0 = timedelta[sorted_indices_0[:10],0]

sorted_indices_1 = list(np.argsort(timedelta[:,1]))[::-1]
cell_fil_1 = np.array(cell[1])[sorted_indices_1[:10]]
timestamps_1 = timedelta[sorted_indices_1[:10],1]


for i in range(len(cell_fil_0)):
    print(cell_fil_0[i], timestamps_0[i], cell_fil_1[i], timestamps_1[i])


60596 5.916876 54814 5.9744453
35611 5.7858233 35346 5.788532
61112 4.808791 60592 5.641257
54069 3.508556 35348 5.097119
35348 3.2877798 35083 4.0067267
35346 3.0500627 34821 2.8439398
40877 2.77363 60596 2.8134108
36408 2.7223914 35084 2.7547216
33268 2.6270812 35084 2.7013612
30617 2.5310478 35347 2.6810215


In [87]:
print(sorted_indices_0)
print(sorted_indices_1)

[4, 10, 5, 2, 9, 11, 8, 12, 13, 14, 7, 6, 3, 0, 1, 21, 15, 16, 17, 18, 19, 20, 22]
[3, 15, 5, 21, 17, 18, 4, 19, 12, 16, 13, 20, 11, 0, 2, 7, 8, 1, 10, 6, 9, 14, 22]


In [61]:
for i in range(timedelta.shape[0]):
    print( cell[0][i], timedelta[i][0], math.exp(timedelta[i][0]))
    print(cell[1][i], timedelta[i][1], math.exp(timedelta[i][1]))
    print("----")

35349 tensor(2.4610, device='cuda:0') 11.716667063220463
53258 tensor(2.5802, device='cuda:0') 13.200000726273418
----
38772 tensor(2.4394, device='cuda:0') 11.466666985455259
54025 tensor(2.4054, device='cuda:0') 11.083333309380361
----
44564 tensor(2.4582, device='cuda:0') 11.683332592622698
61112 tensor(6.5652, device='cuda:0') 709.9666320382654
----
54814 tensor(5.9744, device='cuda:0') 393.2499217743996
57967 tensor(2.6684, device='cuda:0') 14.416667879531264
----
60596 tensor(2.8134, device='cuda:0') 16.66666737020221
46141 tensor(2.8054, device='cuda:0') 16.533331547393907
----
60592 tensor(5.6413, device='cuda:0') 281.8166858071736
40877 tensor(2.5802, device='cuda:0') 13.200000726273418
----
59012 tensor(2.4277, device='cuda:0') 11.333332963068083
38246 tensor(2.8497, device='cuda:0') 17.2833334193748
----
50083 tensor(2.4423, device='cuda:0') 11.5000001649552
36405 tensor(5.6676, device='cuda:0') 289.34995466407923
----
44562 tensor(2.4409, device='cuda:0') 11.4833338007613
3

IndexError: list index out of range